<a href="https://colab.research.google.com/github/longnguyenQB/Heart-disease-predict/blob/main/fill_VN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# !git clone https://github.com/attardi/wikiextractor.git


In [ ]:
# pip install wikiextractor

In [ ]:
!python wikiextractor/WikiExtractor.py viwiki-20220401-pages-articles-multistream.xml.bz2 --processes 4 -o ./output/ --json


INFO: Preprocessing 'viwiki-20220401-pages-articles-multistream.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Loaded 206666 templates in 334.2s
INFO: Starting page extraction from viwiki-20220401-pages-articles-multistream.xml.bz2.
INFO: Using 4 extract processes.
> /content/drive/MyDrive

In [ ]:
pip install unidecode

     |████████████████████████████████| 235 kB 5.3 MB/s 


In [ ]:
import re
import unidecode
import itertools
from nltk import ngrams
import string
import numpy as np
from tqdm import tqdm
import pickle


In [ ]:
# PATH_DATA = "./output"

# alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'

# list_sub_folder = os.listdir(PATH_DATA)
# for sub_folder in (list_sub_folder):
#     path_sub_folder = os.path.join(PATH_DATA, sub_folder)
    
#     list_file = os.listdir(path_sub_folder)
    
#     for file in tqdm(list_file):
#         with open(os.path.join(path_sub_folder, file), "r") as f_r:
#             contents = f_r.read()
#             contents = re.sub("(\s)+", r"\1", contents)
#             contents = contents.split("\n")
#             for content in contents:
#                 try:
#                     content = eval(content)
#                 except:
#                     continue
#                 lines = content["text"].split("\n")
#                 with open("./train_data.txt", "a") as f_w:
#                     for line in lines[1:]:
#                         if len(line.split()) > 2 and re.match(alphabet, line.lower()):
#                             f_w.write(line + "\n")

In [ ]:
# with open("./train_data.txt", "r") as f_r:
#     lines = f_r.read().split("\n")
    
# print(len(lines))


In [ ]:
MAXLEN = 30
NGRAM = 5
BATCH_SIZE = 1024
def remove_accent(text):
    return unidecode.unidecode(text)
def extract_phrases(text):
    return re.findall(r'\w[\w ]+', text)

In [ ]:
# phrases = itertools.chain.from_iterable(extract_phrases(text) for text in lines)
# phrases = [p.strip() for p in phrases if len(p.split()) > 1]


In [ ]:
def gen_ngrams(words, n=5):
    return ngrams(words.split(), n)

In [ ]:
# list_ngrams = []

# for p in tqdm(phrases):
#   for ngr in gen_ngrams(p, NGRAM):
#     if len(" ".join(ngr)) < 30:
#       list_ngrams.append(" ".join(ngr))
# del phrases
# list_ngrams = list(set(list_ngrams))


In [ ]:

with open('list_ngrams.data', 'rb') as filehandle:
    # read the data as binary data stream
    list_ngrams = pickle.load(filehandle)

In [ ]:
accented_chars_vietnamese = [
    'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ',
    'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ',
    'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự',
    'í', 'ì', 'ỉ', 'ĩ', 'ị',
    'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ',
    'đ', 
]
accented_chars_vietnamese.extend([c.upper() for c in accented_chars_vietnamese])
alphabet = list(('\x00 _' + string.ascii_letters + string.digits + ''.join(accented_chars_vietnamese)))


In [ ]:
def encode(text, maxlen=MAXLEN):
        text = "\x00" + text
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x

def decode(x, calc_argmax=True):
    if calc_argmax:
        x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)


In [ ]:
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, LSTM, Bidirectional
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

HIDDEN_SIZE = 256

model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, len(alphabet)), return_sequences=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(len(alphabet))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 256)           466944    
                                                                 
 bidirectional (Bidirectiona  (None, 30, 512)          1050624   
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 30, 199)          102087    
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 30, 199)           0         
                                                                 
Total params: 1,619,655
Trainable params: 1,619,655
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(list_ngrams, test_size=0.4, random_state=2019)


In [ ]:
len(train_data)

25488871

In [ ]:
def generate_data(data, batch_size=128):
    cur_index = 0
    while True:
        
        x, y = [], []
        for i in range(batch_size):  
            y.append(encode(data[cur_index]))
            x.append(encode(unidecode.unidecode(data[cur_index])))
            cur_index += 1
            
            if cur_index > len(data)-1:
                cur_index = 0
        
        yield np.array(x), np.array(y)
    


In [ ]:
import os
train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)

checkpointer = ModelCheckpoint(filepath=os.path.join('./model_{val_loss:.4f}_{val_acc:.4f}.h5'), save_best_only=True, verbose=1)
early = EarlyStopping(patience=2, verbose=1)

model.fit(train_generator, steps_per_epoch=len(train_data)//BATCH_SIZE, epochs=1,
                    validation_data=validation_generator, validation_steps=len(valid_data)//BATCH_SIZE,
                    callbacks=[checkpointer, early])


   19/24891 [..............................] - ETA: 6:12:59 - loss: 3.6698 - accuracy: 0.3091